In [ ]:
# load debate data
import csv
data = (open('resources/QEC Parliament.csv', 'r', encoding='mac_roman'))

### Lexicon approach

In [ ]:
from collections import OrderedDict

debates = csv.reader(data)
data.seek(0) # reset csv reader

# get nested dict with lists of statements by party for each date
debates_over_time = {}
for row in debates:
    if row[-2] not in debates_over_time:
        debates_over_time[row[-2]] = OrderedDict()
    if row[0] not in debates_over_time[row[-2]]:
        debates_over_time[row[-2]][row[0]] = [row[-1]]
    else:
        debates_over_time[row[-2]][row[0]].append(row[-1])

In [ ]:
from nltk import word_tokenize, pos_tag # pos tagging
from nltk.corpus import sentiwordnet as swn
from nltk import download
download('punkt')
download('averaged_perceptron_tagger')
download('sentiwordnet')
download('wordnet')

pos_dict = {'v': 'V', 'n': 'NN', 'r': 'RB', 'a': 'JJ'} # translate between sentiwordnet and nltk pos tag schema


print('Date    | Score | Polarity')

# for each party, for each day/debate, print sentiment score 
for key, val in debates_over_time.items():
    print('\n',key)
    for k, v in val.items():
        score = 0.0
        polarity = ''
        for statement in v:
            for word in pos_tag(word_tokenize(statement)):
                for key, val in pos_dict.items():
                    if val in word[1]:
                        try:
                            score += list(swn.senti_synsets(word[0].lower(),key))[0].pos_score() - \
                                        list(swn.senti_synsets(word[0].lower(),key))[0].neg_score()
                        except:
                            pass
        if score > 0.0:
            polarity = 'Positive'
        elif score < 0.0:
            polarity = 'Negative'
        else:
            polarity = 'Neutral'
        print(k, score, polarity)

### Machine learning approach

In [ ]:
debates = csv.reader(data)
data.seek(0) # reset csv reader

debates_dict = {}

for row in debates:
    date = row[0].split('/')[2] + row[0].split('/')[1] + row[0].split('/')[0]
    if date + ' ' + row[3] not in debates_dict:
        debates_dict[date + ' ' + row[3]] = ['','','', '',
                                             '', row[-1], '', '', '', '',
                                             '', '', row[5], row[3], 0]
    else:
        for i in range(6,10):
            if debates_dict[date + ' ' + row[3]][i] == '':
                debates_dict[date + ' ' + row[3]][i] = row[-1]
                break

debates_dict2 = {}
                
for k, v in debates_dict.items():
    debates_dict2[k] = k + ', ' + str(v)[1:-1]

In [ ]:
# load pretrained ML model
import pickle
with open('resources/pretrainedmodel.pkl', 'rb') as fin:
    vectorizer, SVM = pickle.load(fin)

In [ ]:
for key, val in debates_over_time.items():
    print(key)
    for k, v in val.items():
        speech = ''
        for i in v:
            speech += i + ' '
        X_new = vectorizer.transform([speech])
        prediction = SVM.predict(X_new)[0]
        if prediction == 1:
            polarity = 'Positive'
        else:
            polarity = 'Negative'
        print(k, SVM.predict(X_new)[0], polarity)
    print('\n')

In [ ]:
X_new = vectorizer.transform(["approve great fantastic love", "condemn disgrace awful disappointing"])

In [ ]:
print(SVM.predict(X_new))